In [1]:
import torch

#imports necessary to define a neural network 
import torch.nn as nn
import torch.nn.functional as F

#ensure you are using GPU.
if torch.cuda.is_available():  
  dev = "cuda:0" 
else:  
  dev = "cpu"  

device = torch.device(dev)
print(device)

dtype = torch.cuda.FloatTensor

cuda:0


In [2]:
a0=9.0
a1=8.97
a2=0.876
a3=2.9876

def output(x0,x1,x2,x3):
    
    out=a0*x0**(0.87)
    out=out+a1*x1**(0.020)
    out=out+a2*x2**(0.12)
    out=out+a3*x3**(0.987)
    
    return out

In [3]:
xtrain=1000.0*torch.rand(10000,4).type(dtype)
xtest=1000.0*torch.rand(1000,4).type(dtype)

In [4]:
ytest=torch.empty(1000,1).type(dtype)

for i in range(1000):
    
    ytest[i]=output(xtest[i][0],xtest[i][1],xtest[i][2],xtest[i][3]).type(dtype)

ytrain=torch.empty(10000,1).type(dtype)

for i in range(10000):
    
    ytrain[i]=output(xtrain[i][0],xtrain[i][1],xtrain[i][2],xtrain[i][3]).type(dtype)

In [5]:
class Forward(nn.Module):
    def __init__(self):
    
        super(Forward, self).__init__()


        self.fc1 = nn.Linear(4,80)
        self.fc2 = nn.Linear(80,160)
        self.fc4=nn.Linear(160,16)
        self.fc3=nn.Linear(16,1)
        '''
        
        self.fc3 = nn.Linear(6400, 12800)

        self.fc5=nn.Linear(12800, 12800)
        self.fc6=nn.Linear(12800, 1280)
        self.fc4 = nn.Linear(1280, 784)


        self.dropout = nn.Dropout(0.3)
        '''
    def forward(self, x):
        
        x = F.relu(self.fc1(x)) # (input, negative_slope=0.2)
        
        x = F.relu(self.fc2(x))
        
        x = F.relu(self.fc4(x))
        
        x = F.relu(self.fc3(x))
        
        '''
        out =F.relu(self.fc5(x))
        out =F.relu(self.fc6(out))
        out =F.relu(self.fc4(out))
        out[out>255]=out[out>255]/100
        out[out<55]=out[out<55]*1000
        '''
        return x

In [6]:
import torch.optim as optim

learning_rate=0.001
momentum=0.05

forwardnetwork = Forward().cuda()
forwardoptimizer = optim.Adagrad(forwardnetwork.parameters(), lr=learning_rate,
                      )
print(forwardnetwork)

Forward(
  (fc1): Linear(in_features=4, out_features=80, bias=True)
  (fc2): Linear(in_features=80, out_features=160, bias=True)
  (fc4): Linear(in_features=160, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=1, bias=True)
)


In [7]:
forwardnetwork(xtrain[0:10].reshape(10,4))

tensor([[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]], device='cuda:0', grad_fn=<ReluBackward0>)

In [8]:
#training loop

batchsize=8

numbatches=int(len(xtrain)/batchsize)

nepochs=1000
criterion=nn.MSELoss()

for i in range(nepochs):
    
    epochloss=0
    
    for j in range(numbatches):
        
        out=forwardnetwork(xtrain[j*batchsize:(j+1)*batchsize].reshape(batchsize,4)).type(dtype)
        out=out.reshape(batchsize,1)
        #print(out)
        loss=criterion(out,ytrain[j*batchsize:(j+1)*batchsize].reshape(batchsize,1)).type(dtype)
        
        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(forwardnetwork.parameters(), 1.0)
        forwardoptimizer.step()
        
        epochloss=epochloss+loss
        
        testout=forwardnetwork(xtest.reshape(1000,4))
        lossc=nn.MSELoss()
        testloss=lossc(testout,ytest)
        
    print(epochloss,testloss,i)

tensor(1.1899e+10, device='cuda:0', grad_fn=<AddBackward0>) tensor(764001.4375, device='cuda:0', grad_fn=<MseLossBackward>) 0
tensor(1.7981e+08, device='cuda:0', grad_fn=<AddBackward0>) tensor(12750.1729, device='cuda:0', grad_fn=<MseLossBackward>) 1
tensor(12365383., device='cuda:0', grad_fn=<AddBackward0>) tensor(7404.8755, device='cuda:0', grad_fn=<MseLossBackward>) 2
tensor(8712702., device='cuda:0', grad_fn=<AddBackward0>) tensor(5953.1421, device='cuda:0', grad_fn=<MseLossBackward>) 3
tensor(7252744.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(5277.3252, device='cuda:0', grad_fn=<MseLossBackward>) 4
tensor(6469716.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(4880.4277, device='cuda:0', grad_fn=<MseLossBackward>) 5
tensor(5962502., device='cuda:0', grad_fn=<AddBackward0>) tensor(4602.4531, device='cuda:0', grad_fn=<MseLossBackward>) 6
tensor(5599654.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(4391.9697, device='cuda:0', grad_fn=<MseLossBackward>) 7
tens

tensor(3747522.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3141.1545, device='cuda:0', grad_fn=<MseLossBackward>) 66
tensor(3743921., device='cuda:0', grad_fn=<AddBackward0>) tensor(3138.1526, device='cuda:0', grad_fn=<MseLossBackward>) 67
tensor(3740357., device='cuda:0', grad_fn=<AddBackward0>) tensor(3135.2493, device='cuda:0', grad_fn=<MseLossBackward>) 68
tensor(3736879.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(3132.4128, device='cuda:0', grad_fn=<MseLossBackward>) 69
tensor(3733442.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(3129.6038, device='cuda:0', grad_fn=<MseLossBackward>) 70
tensor(3730077.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3126.8379, device='cuda:0', grad_fn=<MseLossBackward>) 71
tensor(3726741., device='cuda:0', grad_fn=<AddBackward0>) tensor(3124.1584, device='cuda:0', grad_fn=<MseLossBackward>) 72
tensor(3723525.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3121.3821, device='cuda:0', grad_fn=<MseLossBackward>)

tensor(3595684.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3012.3462, device='cuda:0', grad_fn=<MseLossBackward>) 131
tensor(3594028.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3010.9426, device='cuda:0', grad_fn=<MseLossBackward>) 132
tensor(3592359.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3009.5459, device='cuda:0', grad_fn=<MseLossBackward>) 133
tensor(3590728.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3008.1311, device='cuda:0', grad_fn=<MseLossBackward>) 134
tensor(3589099.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(3006.7065, device='cuda:0', grad_fn=<MseLossBackward>) 135
tensor(3587476.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(3005.3984, device='cuda:0', grad_fn=<MseLossBackward>) 136
tensor(3585846.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3004.0562, device='cuda:0', grad_fn=<MseLossBackward>) 137
tensor(3584222.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(3002.7246, device='cuda:0', grad_fn

tensor(3503675.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2937.7417, device='cuda:0', grad_fn=<MseLossBackward>) 196
tensor(3502493., device='cuda:0', grad_fn=<AddBackward0>) tensor(2936.8201, device='cuda:0', grad_fn=<MseLossBackward>) 197
tensor(3501296.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2935.8806, device='cuda:0', grad_fn=<MseLossBackward>) 198
tensor(3500115.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(2934.9956, device='cuda:0', grad_fn=<MseLossBackward>) 199
tensor(3498946.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2934.0767, device='cuda:0', grad_fn=<MseLossBackward>) 200
tensor(3497782.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2933.1487, device='cuda:0', grad_fn=<MseLossBackward>) 201
tensor(3496624.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2932.2590, device='cuda:0', grad_fn=<MseLossBackward>) 202
tensor(3495461.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2931.3286, device='cuda:0', grad_fn=<Ms

tensor(3435300., device='cuda:0', grad_fn=<AddBackward0>) tensor(2884.5803, device='cuda:0', grad_fn=<MseLossBackward>) 261
tensor(3434351.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(2883.8308, device='cuda:0', grad_fn=<MseLossBackward>) 262
tensor(3433405.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2883.1069, device='cuda:0', grad_fn=<MseLossBackward>) 263
tensor(3432480.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2882.3887, device='cuda:0', grad_fn=<MseLossBackward>) 264
tensor(3431536.2500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2881.6838, device='cuda:0', grad_fn=<MseLossBackward>) 265
tensor(3430624.5000, device='cuda:0', grad_fn=<AddBackward0>) tensor(2880.9775, device='cuda:0', grad_fn=<MseLossBackward>) 266
tensor(3429683.7500, device='cuda:0', grad_fn=<AddBackward0>) tensor(2880.2585, device='cuda:0', grad_fn=<MseLossBackward>) 267
tensor(3428749., device='cuda:0', grad_fn=<AddBackward0>) tensor(2879.5647, device='cuda:0', grad_fn=<MseLos

KeyboardInterrupt: 

In [75]:
torch.save(forwardnetwork, '/home/Downloads')

PermissionError: [Errno 13] Permission denied: '/home/Downloads'

In [33]:
xtes=torch.rand(10,4).type(dtype)
ypred=forwardnetwork(xtes)
yreal=output(xtes[0][0],xtes[0][1],xtes[0][2],xtes[0][3])

print(xtes)
print(ypred)
print(yreal)
print(output(xtes[1][0],xtes[1][1],xtes[1][2],xtes[1][3]))

print(output(xtes[2][0],xtes[2][1],xtes[2][2],xtes[2][3]))

print(output(xtes[3][0],xtes[3][1],xtes[3][2],xtes[3][3]))

tensor([[0.8535, 0.1850, 0.7229, 0.1189],
        [0.5709, 0.0867, 0.1855, 0.7886],
        [0.2509, 0.1231, 0.3228, 0.9881],
        [0.9705, 0.1609, 0.1677, 0.7459],
        [0.9030, 0.3394, 0.5157, 0.2805],
        [0.6119, 0.3524, 0.0762, 0.1286],
        [0.1373, 0.7849, 0.5199, 0.3923],
        [0.2094, 0.0311, 0.8163, 0.6292],
        [0.6515, 0.1506, 0.2137, 0.1372],
        [0.0288, 0.0982, 0.1838, 0.5902]], device='cuda:0')
tensor([[17.6763],
        [18.5846],
        [17.7937],
        [19.9657],
        [18.4549],
        [16.7925],
        [15.6475],
        [16.3899],
        [16.9381],
        [15.5786]], device='cuda:0', grad_fn=<ReluBackward0>)
tensor(17.7211, device='cuda:0')
tensor(17.1469, device='cuda:0')
tensor(15.0220, device='cuda:0')
tensor(20.3606, device='cuda:0')


In [55]:
class Generator(nn.Module):
    def __init__(self):
    
        super(Generator, self).__init__()


        self.fc1 = nn.Linear(40,80)
        self.fc2 = nn.Linear(80,16)
        self.fc4=nn.Linear(16,16)
        self.fc3=nn.Linear(16,3)
        '''
        
        self.fc3 = nn.Linear(6400, 12800)

        self.fc5=nn.Linear(12800, 12800)
        self.fc6=nn.Linear(12800, 1280)
        self.fc4 = nn.Linear(1280, 784)


        self.dropout = nn.Dropout(0.3)
        '''
    def forward(self, x):
        
        x = F.relu(self.fc1(x)) # (input, negative_slope=0.2)
        
        x = F.relu(self.fc2(x))
        
        x = F.relu(self.fc4(x))
        
        x = F.relu(self.fc3(x))
        
        '''
        out =F.relu(self.fc5(x))
        out =F.relu(self.fc6(out))
        out =F.relu(self.fc4(out))
        out[out>255]=out[out>255]/100
        out[out<55]=out[out<55]*1000
        '''
        return x

In [56]:
gennetwork = Generator().cuda()
genoptimizer = optim.Adagrad(gennetwork.parameters(), lr=learning_rate,
                            )
print(gennetwork)

Generator(
  (fc1): Linear(in_features=40, out_features=80, bias=True)
  (fc2): Linear(in_features=80, out_features=16, bias=True)
  (fc4): Linear(in_features=16, out_features=16, bias=True)
  (fc3): Linear(in_features=16, out_features=3, bias=True)
)


In [57]:
def customloss(inp,bsize):
    
    inp=torch.cat((inp,10*torch.ones(bsize,1).type(dtype)),1).type(dtype)
    cusout=forwardnetwork(inp.reshape([bsize,4])).type(dtype)
    
    cusout=cusout.reshape([bsize,1]).type(dtype)
    cusideal=2000*torch.ones([bsize,1]).type(dtype)
    
    cuscriterion= nn.MSELoss()
    
    loss= cuscriterion(cusout,cusideal).type(dtype)
    
    return loss

In [58]:
xgarb=torch.rand([100,40]).type(dtype)

In [62]:
batchsize=2

batches=len(xgarb)/batchsize

epochs=1000

for i in range(epochs):
    
    epochloss=0
        
    for j in range(int(batches)):
        
        
        #print(Ytensor)
        out=gennetwork(xgarb[j*batchsize:(j+1)*batchsize].reshape([batchsize,40])).type(dtype)
        
        #compute loss
        loss = customloss(out,batchsize).type(dtype)


        #backprop loss i.e. find dloss/dparam for each parameter and store.
        loss.backward(retain_graph=True)
        
        #clip gradients
        torch.nn.utils.clip_grad_norm_(gennetwork.parameters(), 1.0)
        genoptimizer.step()
        
        epochloss=epochloss+loss
        
    print(epochloss,i)

tensor(1.8593e+08, device='cuda:0', grad_fn=<AddBackward0>) 0
tensor(1.8561e+08, device='cuda:0', grad_fn=<AddBackward0>) 1
tensor(1.8528e+08, device='cuda:0', grad_fn=<AddBackward0>) 2
tensor(1.8495e+08, device='cuda:0', grad_fn=<AddBackward0>) 3
tensor(1.8460e+08, device='cuda:0', grad_fn=<AddBackward0>) 4
tensor(1.8425e+08, device='cuda:0', grad_fn=<AddBackward0>) 5
tensor(1.8389e+08, device='cuda:0', grad_fn=<AddBackward0>) 6
tensor(1.8352e+08, device='cuda:0', grad_fn=<AddBackward0>) 7
tensor(1.8314e+08, device='cuda:0', grad_fn=<AddBackward0>) 8
tensor(1.8276e+08, device='cuda:0', grad_fn=<AddBackward0>) 9
tensor(1.8237e+08, device='cuda:0', grad_fn=<AddBackward0>) 10
tensor(1.8198e+08, device='cuda:0', grad_fn=<AddBackward0>) 11
tensor(1.8159e+08, device='cuda:0', grad_fn=<AddBackward0>) 12
tensor(1.8118e+08, device='cuda:0', grad_fn=<AddBackward0>) 13
tensor(1.8078e+08, device='cuda:0', grad_fn=<AddBackward0>) 14
tensor(1.8036e+08, device='cuda:0', grad_fn=<AddBackward0>) 15
te

tensor(1.0998e+08, device='cuda:0', grad_fn=<AddBackward0>) 130
tensor(1.0926e+08, device='cuda:0', grad_fn=<AddBackward0>) 131
tensor(1.0853e+08, device='cuda:0', grad_fn=<AddBackward0>) 132
tensor(1.0780e+08, device='cuda:0', grad_fn=<AddBackward0>) 133
tensor(1.0708e+08, device='cuda:0', grad_fn=<AddBackward0>) 134
tensor(1.0635e+08, device='cuda:0', grad_fn=<AddBackward0>) 135
tensor(1.0562e+08, device='cuda:0', grad_fn=<AddBackward0>) 136
tensor(1.0489e+08, device='cuda:0', grad_fn=<AddBackward0>) 137
tensor(1.0416e+08, device='cuda:0', grad_fn=<AddBackward0>) 138
tensor(1.0344e+08, device='cuda:0', grad_fn=<AddBackward0>) 139
tensor(1.0271e+08, device='cuda:0', grad_fn=<AddBackward0>) 140
tensor(1.0198e+08, device='cuda:0', grad_fn=<AddBackward0>) 141
tensor(1.0125e+08, device='cuda:0', grad_fn=<AddBackward0>) 142
tensor(1.0052e+08, device='cuda:0', grad_fn=<AddBackward0>) 143
tensor(99789088., device='cuda:0', grad_fn=<AddBackward0>) 144
tensor(99059432., device='cuda:0', grad_f

tensor(24114712., device='cuda:0', grad_fn=<AddBackward0>) 260
tensor(23622958., device='cuda:0', grad_fn=<AddBackward0>) 261
tensor(23135476., device='cuda:0', grad_fn=<AddBackward0>) 262
tensor(22652278., device='cuda:0', grad_fn=<AddBackward0>) 263
tensor(22173414., device='cuda:0', grad_fn=<AddBackward0>) 264
tensor(21698932., device='cuda:0', grad_fn=<AddBackward0>) 265
tensor(21228884., device='cuda:0', grad_fn=<AddBackward0>) 266
tensor(20763284., device='cuda:0', grad_fn=<AddBackward0>) 267
tensor(20302200., device='cuda:0', grad_fn=<AddBackward0>) 268
tensor(19845668., device='cuda:0', grad_fn=<AddBackward0>) 269
tensor(19393714., device='cuda:0', grad_fn=<AddBackward0>) 270
tensor(18946382., device='cuda:0', grad_fn=<AddBackward0>) 271
tensor(18503724., device='cuda:0', grad_fn=<AddBackward0>) 272
tensor(18065788., device='cuda:0', grad_fn=<AddBackward0>) 273
tensor(17632598., device='cuda:0', grad_fn=<AddBackward0>) 274
tensor(17204210., device='cuda:0', grad_fn=<AddBackward

tensor(1637938., device='cuda:0', grad_fn=<AddBackward0>) 388
tensor(1637260.7500, device='cuda:0', grad_fn=<AddBackward0>) 389
tensor(1636593.5000, device='cuda:0', grad_fn=<AddBackward0>) 390
tensor(1635940.6250, device='cuda:0', grad_fn=<AddBackward0>) 391
tensor(1635301., device='cuda:0', grad_fn=<AddBackward0>) 392
tensor(1634673.7500, device='cuda:0', grad_fn=<AddBackward0>) 393
tensor(1634056.6250, device='cuda:0', grad_fn=<AddBackward0>) 394
tensor(1633449.1250, device='cuda:0', grad_fn=<AddBackward0>) 395
tensor(1632852.2500, device='cuda:0', grad_fn=<AddBackward0>) 396
tensor(1632261.5000, device='cuda:0', grad_fn=<AddBackward0>) 397
tensor(1631679.6250, device='cuda:0', grad_fn=<AddBackward0>) 398
tensor(1631102.7500, device='cuda:0', grad_fn=<AddBackward0>) 399
tensor(1630532.8750, device='cuda:0', grad_fn=<AddBackward0>) 400
tensor(1629967.1250, device='cuda:0', grad_fn=<AddBackward0>) 401
tensor(1629407.6250, device='cuda:0', grad_fn=<AddBackward0>) 402
tensor(1628852.875

tensor(1581832.2500, device='cuda:0', grad_fn=<AddBackward0>) 514
tensor(1581491.5000, device='cuda:0', grad_fn=<AddBackward0>) 515
tensor(1581152.1250, device='cuda:0', grad_fn=<AddBackward0>) 516
tensor(1580811.6250, device='cuda:0', grad_fn=<AddBackward0>) 517
tensor(1580473.8750, device='cuda:0', grad_fn=<AddBackward0>) 518
tensor(1580136.3750, device='cuda:0', grad_fn=<AddBackward0>) 519
tensor(1579800.6250, device='cuda:0', grad_fn=<AddBackward0>) 520
tensor(1579466.2500, device='cuda:0', grad_fn=<AddBackward0>) 521
tensor(1579130.8750, device='cuda:0', grad_fn=<AddBackward0>) 522
tensor(1578796., device='cuda:0', grad_fn=<AddBackward0>) 523
tensor(1578461.3750, device='cuda:0', grad_fn=<AddBackward0>) 524
tensor(1578127.3750, device='cuda:0', grad_fn=<AddBackward0>) 525
tensor(1577794.2500, device='cuda:0', grad_fn=<AddBackward0>) 526
tensor(1577462.6250, device='cuda:0', grad_fn=<AddBackward0>) 527
tensor(1577132.7500, device='cuda:0', grad_fn=<AddBackward0>) 528
tensor(1576803

tensor(1544572.7500, device='cuda:0', grad_fn=<AddBackward0>) 640
tensor(1544309.7500, device='cuda:0', grad_fn=<AddBackward0>) 641
tensor(1544062.3750, device='cuda:0', grad_fn=<AddBackward0>) 642
tensor(1543807.5000, device='cuda:0', grad_fn=<AddBackward0>) 643
tensor(1543553., device='cuda:0', grad_fn=<AddBackward0>) 644
tensor(1543299.1250, device='cuda:0', grad_fn=<AddBackward0>) 645
tensor(1543046.3750, device='cuda:0', grad_fn=<AddBackward0>) 646
tensor(1542793.7500, device='cuda:0', grad_fn=<AddBackward0>) 647
tensor(1542541.6250, device='cuda:0', grad_fn=<AddBackward0>) 648
tensor(1542290.7500, device='cuda:0', grad_fn=<AddBackward0>) 649
tensor(1542038.2500, device='cuda:0', grad_fn=<AddBackward0>) 650
tensor(1541788.6250, device='cuda:0', grad_fn=<AddBackward0>) 651
tensor(1541537.5000, device='cuda:0', grad_fn=<AddBackward0>) 652
tensor(1541287.1250, device='cuda:0', grad_fn=<AddBackward0>) 653
tensor(1541036.1250, device='cuda:0', grad_fn=<AddBackward0>) 654
tensor(1540787

tensor(1515572.2500, device='cuda:0', grad_fn=<AddBackward0>) 766
tensor(1515360.5000, device='cuda:0', grad_fn=<AddBackward0>) 767
tensor(1515151.6250, device='cuda:0', grad_fn=<AddBackward0>) 768
tensor(1514940.8750, device='cuda:0', grad_fn=<AddBackward0>) 769
tensor(1514727.5000, device='cuda:0', grad_fn=<AddBackward0>) 770
tensor(1514522.5000, device='cuda:0', grad_fn=<AddBackward0>) 771
tensor(1514311.1250, device='cuda:0', grad_fn=<AddBackward0>) 772
tensor(1514101.3750, device='cuda:0', grad_fn=<AddBackward0>) 773
tensor(1513891.3750, device='cuda:0', grad_fn=<AddBackward0>) 774
tensor(1513684., device='cuda:0', grad_fn=<AddBackward0>) 775
tensor(1513473.8750, device='cuda:0', grad_fn=<AddBackward0>) 776
tensor(1513266.5000, device='cuda:0', grad_fn=<AddBackward0>) 777
tensor(1513058.7500, device='cuda:0', grad_fn=<AddBackward0>) 778
tensor(1512849.8750, device='cuda:0', grad_fn=<AddBackward0>) 779
tensor(1512642.8750, device='cuda:0', grad_fn=<AddBackward0>) 780
tensor(1512434

tensor(1490643.1250, device='cuda:0', grad_fn=<AddBackward0>) 892
tensor(1490460.7500, device='cuda:0', grad_fn=<AddBackward0>) 893
tensor(1490274.5000, device='cuda:0', grad_fn=<AddBackward0>) 894
tensor(1490088.1250, device='cuda:0', grad_fn=<AddBackward0>) 895
tensor(1489900.8750, device='cuda:0', grad_fn=<AddBackward0>) 896
tensor(1489715.1250, device='cuda:0', grad_fn=<AddBackward0>) 897
tensor(1489528., device='cuda:0', grad_fn=<AddBackward0>) 898
tensor(1489343.3750, device='cuda:0', grad_fn=<AddBackward0>) 899
tensor(1489158.2500, device='cuda:0', grad_fn=<AddBackward0>) 900
tensor(1488975.7500, device='cuda:0', grad_fn=<AddBackward0>) 901
tensor(1488788.7500, device='cuda:0', grad_fn=<AddBackward0>) 902
tensor(1488600.7500, device='cuda:0', grad_fn=<AddBackward0>) 903
tensor(1488419.2500, device='cuda:0', grad_fn=<AddBackward0>) 904
tensor(1488233.7500, device='cuda:0', grad_fn=<AddBackward0>) 905
tensor(1488048.7500, device='cuda:0', grad_fn=<AddBackward0>) 906
tensor(1487863

In [63]:
loss

tensor(32600.3125, device='cuda:0', grad_fn=<MseLossBackward>)

In [67]:
print(out)
print(output(out[0][0],out[0][1],out[0][2],10*torch.ones(1).type(dtype)))

tensor([[485.6413,  70.6677,  90.4546],
        [577.7753,  84.0412, 107.5973]], device='cuda:0',
       grad_fn=<ReluBackward0>)
tensor([1996.1234], device='cuda:0', grad_fn=<AddBackward0>)


In [ ]:

xtens0=torch.linspace(0,10,1000).type(dtype)
xtens1=torch.linspace(2,54,1000).type(dtype)
xtens2=torch.linspace(1,100,1000).type(dtype)
xtens3=torch.linspace(10,100,1000).type(dtype)

ytens=torch.empty(1000,1).type(dtype)

for i in range(1000):
    
    ytens[i]=out(xtens0[i],xtens1[i],xtens2[i],xtens3[i]).type(dtype)

print(ytens)